# Mine Safety

We're interested in [US mine safety](https://arlweb.msha.gov/drs/drshome.htm), thank goodness we can search for these things.

## Setup: Import what you'll need to search and scrape and Selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

## Starting from `https://arlweb.msha.gov/drs/drshome.htm`, search for every operator with 'dirt' in their name, including abandoned mines.

> - *Tip: If you can't make an element work using name, class or ID, try to use the XPath*

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://arlweb.msha.gov/drs/drshome.htm')
operator_name = driver.find_element_by_name('OperSearch')
operator_name.send_keys('dirt')
abandoned = driver.find_element_by_name('Abandoned')
abandoned.click()
search = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table/tbody/tr[7]/td[3]/input[1]')
driver.execute_script('arguments[0].scrollIntoView(true)', search)
search.click()

## Scrape the results page, saving it as `dirt-operators.csv`

> - *Tip: Think about what each row in your dataset will be, and start by looping through that*
> - *Tip: Printing is cool and good! Print everything! Move it into a dictionary later.*
> - *Tip: If you don't want a row, think about what's in the row that makes it different. You can use an `if` statement or list slicing to skip the ones you aren't interested in.*
> - *Tip: Make sure your dictionary and your loop variable have DIFFERENT NAMES*
> - *Tip: After you've made your dictionary (and printed it, of course), you'll want to add it to your list of rows*
> - *Tip: Be sure to import pandas to convert it to a dataframe*
> - *Tip: Make sure you don't include the index when saving your dataframe*

### Hopefully you know that each `tr` is supposed to be a row of your data. What is the index of the first row element that is actually a result?

> - *Tip: `.text` will help you here.*
> - *Tip: You aren't interesting in annotations or anything, just mines and where they are from*
> - *Tip: Using `print("-----")` will help you keep track of different rows*
> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third. You can use this to skip ahead to the 'good' data if you want*

In [4]:
driver.find_elements_by_tag_name('tr')[7].text

'3503598\nOR  Newberg Rock & Dirt   Newberg Rock & Dirt Surface M  Active  Crushed, Broken Stone NEC '

### Loop through each operator result, printing its name

> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third.*
> - *Tip: You can use list slicing or an `if` statement to skip the non-data row(s). List slicing is probably easier, even if you aren't comfortable with it.*
> - *Tip: or honestly you can use `try` and `except` if you know how it works.*
> - *Tip: Once you have the "right" rows of data, you're going to be looking for a certain tag inside*
> - *Tip: Sometimes you can't say "give me this class," and instead you have to say "give me all of the `div` elements, and then give me the third one."*

In [5]:
results = driver.find_elements_by_tag_name('tr')[7:-1]

In [6]:
for result in results:
    print('-------')
    operator = result.find_elements_by_tag_name('font')[2]
    print(operator.text)

-------
Newberg Rock & Dirt  
-------
Allied Dirt Moving Company  
-------
AM Dirtworks & Aggregate Sales  
-------
Atlas-Dirty Devil Mining  
-------
Atlas-Dirty Devil Mining  
-------
Babe's Dirt Work  
-------
Bar-Lin Dirt Company  
-------
Barber'S Dirt Pit  
-------
Bender Sand & Dirt  
-------
BERT'S DIRT  
-------
Big D Dirt Service Inc  
-------
Big Red Dirt Farm LLC  
-------
Big River Dirt Pit  
-------
Bob Harris Dirt Contracting  
-------
Bohannon Sand & Dirt  
-------
Bratcher'S Sand & Dirt  
-------
Brewer Dirt Works  
-------
Buck'S Dirt Pit  
-------
C & G Dirt Hauling  
-------
C N C Dirt Movers, Inc.  
-------
Cambridge Dirt Sand and Gravel LLC  
-------
Central Iowa Dirt & Demo LLC  
-------
Crowes Trucking & Dirt Pit Services  
-------
D & H Dirt  
-------
Diez Dirt & Sand Hauling Inc  
-------
Dirt Cheap  
-------
Dirt Company  
-------
Dirt Company  
-------
Dirt Company  
-------
Dirt Con  
-------
Dirt Diggers Inc  
-------
Dirt Doctor Inc  
-------
Dirt Inc  
-

### Loop through each operator result, printing its ID

There should be ONE code per row, and NO empty rows between them.

In [7]:
for result in results:
    print('-------')
    mineid = result.find_element_by_name('MineId')
    print(mineid.get_attribute('value'))

-------
3503598
-------
0502030
-------
4801789
-------
4201449
-------
4201450
-------
1002257
-------
1601167
-------
4103265
-------
1401575
-------
1700776
-------
1601251
-------
0301963
-------
1601082
-------
3401751
-------
1600916
-------
3401211
-------
0301267
-------
1600956
-------
2200033
-------
0504953
-------
3401929
-------
1302445
-------
1601106
-------
3400915
-------
1600983
-------
4503200
-------
3401266
-------
3401468
-------
5001797
-------
4608254
-------
1510279
-------
2103723
-------
0100776
-------
4104016
-------
2103914
-------
4104757
-------
0301729
-------
0404851
-------
2200734
-------
5002028
-------
1513393
-------
3800602
-------
3101630
-------
3200860
-------
3401762
-------
2103517
-------
2402626
-------
2103181
-------
1601124
-------
1601150
-------
4703427
-------
0801306
-------
2501216
-------
3200965
-------
2901371
-------
2901544
-------
2901709
-------
4102355
-------
4102420
-------
4102869
-------
4102951
-------
4102958
-------


## Saving the results

### Loop through each `tr` to create a list of dictionaries

Each dictionary must contain

- Operator ID
- Operator name
- Mine name
- State
- Mine type
- Coal or metal
- Status
- Commodity

Create a new dictionary for each row.

> - *Tip: Start with an empty dictionary, then add the keys one at a time like we did during class*
> - *Tip: You might want to save all of the cells in a variable, then use indexes to get the second, third, fourth, etc.*
> - *Tip: I know you already skipped a bunch of rows already, but one of them still might be bad! Which one is it? How can you skip it? You might need to slice out some of the end of your list, too. Use `print` to help you debug, or just look at the page closely.*
> - *Tip: Or, if you did the other homework already, `try` / `except` is also an option*

In [8]:
# The ID is not Operator ID actually, but Mine ID.

In [9]:
data = []
for result in results:
    row = {}
    mineid = result.find_element_by_name('MineId')
    state = result.find_element_by_tag_name('b')
    fonts = result.find_elements_by_tag_name('font')
    operator = fonts[2]
    mine = fonts[3]
    minetype = fonts[4]
    cm = fonts[5]
    status = fonts[6]
    commodity = fonts[7]
    row.update({
        'Mine ID': mineid.get_attribute('value'),
        'Operator name': operator.text,
        'Mine name': mine.text,
        'State': state.text,
        'Mine type': minetype.text,
        'Coal or metal': cm.text,
        'Status': status.text,
        'Commodity': commodity.text
    })
    data.append(row)

### Save that to a CSV named `dirt-operators.csv`

In [10]:
df = pd.DataFrame(data)

In [11]:
df.to_csv('dirt-operators.csv', index=False)

### Open the CSV file and examine the first few.

Make sure you didn't save that extra weird unnamed index column.

In [12]:
pd.read_csv('dirt-operators.csv').head()

,Coal or metal,Commodity,Mine ID,Mine name,Mine type,Operator name,State,Status
0,M,"Crushed, Broken Stone NEC",3503598,Newberg Rock & Dirt,Surface,Newberg Rock & Dirt,OR,Active
1,M,Construction Sand and Gravel,502030,Allied Dirt Moving Co Pit & Plant,Surface,Allied Dirt Moving Company,CO,Abandoned
2,M,Construction Sand and Gravel,4801789,AM Dirtworks & Aggregate Sales,Surface,AM Dirtworks & Aggregate Sales,ND,Abandoned
3,C,Coal (Bituminous),4201449,Unit Train Loading Facility,Facility,Atlas-Dirty Devil Mining,UT,Abandoned
4,C,Coal (Bituminous),4201450,Blackie Surface Mine & Prep Plant,Surface,Atlas-Dirty Devil Mining,UT,Abandoned
